In [ ]:
!pip install sqlalchemy

In [1]:
from sqlalchemy import create_engine

In [2]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2

In [3]:
con = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")

In [ ]:
cur = con.cursor()
sql = "SELECT * from passengers"
dat1 = sqlio.read_sql_query(sql, con)


In [5]:
dat1.to_csv('../data/passengers.csv')

In [25]:
# Getting the data for the top 10 airports and their number of destinations in order
cur = con.cursor()
sql = "SELECT COUNT(dest_airport_id) AS dest_count, origin_airport_id  FROM flights GROUP BY origin_airport_id ORDER BY dest_count DESC LIMIT 10"
dat_top_origins = sqlio.read_sql_query(sql, con)

In [27]:
dat_top_origins = pd.read_csv('../data/top_origin_airport.csv')

In [28]:
dat_top_origins

,Unnamed: 0,dest_count,origin_airport_id
0,0,790253,13930
1,1,785850,10397
2,2,584688,11298
3,3,552134,11292
4,4,501557,11057
5,5,500840,12892
6,6,379367,14747
7,7,357626,14771
8,8,355485,14107
9,9,354379,12266


In [ ]:
# Getting all of the flight data for the top 10 airports
sql = "SELECT * FROM flights WHERE origin_airport_id IN (13930,10397,11298,11292,11057,12892,14747,14771,14107,12266)"
dat_flights_by_origins = sqlio.read_sql_query(sql, con)

In [ ]:
# Getting the unique carriers for these airports
sql = "SELECT DISTINCT mkt_carrier FROM flights"
dat3 = sqlio.read_sql_query(sql, con)

In [13]:
df_passengers = pd.read_csv('../data/passengers.csv')

In [18]:
df_passengers = df_passengers.drop(['Unnamed: 0'], axis = 1)

In [19]:
df_passengers.columns

Index(['departures_scheduled', 'departures_performed', 'payload', 'seats',
       'passengers', 'freight', 'mail', 'distance', 'ramp_to_ramp', 'air_time',
       'unique_carrier', 'airline_id', 'unique_carrier_name', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_airport_id', 'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_country', 'origin_country_name',
       'dest_airport_id', 'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_country', 'dest_country_name', 'aircraft_group', 'aircraft_type',
       'aircraft_config', 'year', 'month', 'distance_group', 'class',
       'data_source'],
      dtype='object')

## Checking Passengers DataFrame

In [84]:
df_pass = pd.read_csv('../data/passengers.csv')

In [ ]:
sql = "SELECT * FROM passengers WHERE origin_airport_id IN (13930,10397,11298,11292,11057,12892,14747,14771,14107,12266)"
df_pass_origin = sqlio.read_sql_query(sql, con)
df_pass_origin.to_csv('../data/pass_top_origin.csv')

In [ ]:
df_pass_origin = df_pass.loc[(df_pass['year'] == '2018') | (df_pass['year'] == '2019')]

In [5]:
df_pass_origin.columns

Index(['departures_scheduled', 'departures_performed', 'payload', 'seats',
       'passengers', 'freight', 'mail', 'distance', 'ramp_to_ramp', 'air_time',
       'unique_carrier', 'airline_id', 'unique_carrier_name', 'region',
       'carrier', 'carrier_name', 'carrier_group', 'carrier_group_new',
       'origin_airport_id', 'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_country', 'origin_country_name',
       'dest_airport_id', 'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_country', 'dest_country_name', 'aircraft_group', 'aircraft_type',
       'aircraft_config', 'year', 'month', 'distance_group', 'class',
       'data_source'],
      dtype='object')

In [164]:
# selecting relevant columns
df_pass = df_pass_origin[['origin_airport_id','month', 'year', 
                          'unique_carrier', 'unique_carrier_name', 
                          'dest_airport_id','departures_performed', 
                          'payload', 'seats', 'passengers', 
                          'freight', 'mail', 'ramp_to_ramp',
                           'region', 'carrier_group_new', 
                           'aircraft_group', 'aircraft_type', 'aircraft_config', 
                           'distance_group', 'class',
                           'data_source']]

In [165]:
df_pass = df_pass.sort_values(['origin_airport_id','dest_airport_id', 'unique_carrier','year','month'])

In [167]:
pass_cleaned = df_pass.copy()

In [168]:
pass_cleaned['payload_per_trip'] = pass_cleaned['payload']/pass_cleaned['departures_performed']
pass_cleaned['seats_per_trip'] = pass_cleaned['seats']/pass_cleaned['departures_performed']
pass_cleaned['passengers_per_trip'] = pass_cleaned['passengers']/pass_cleaned['departures_performed']

In [170]:
pass_cleaned.isnull().sum()

origin_airport_id         0
month                     0
year                      0
unique_carrier            0
unique_carrier_name       0
dest_airport_id           0
departures_performed      0
payload                   0
seats                     0
passengers                0
freight                   0
mail                      0
ramp_to_ramp              0
region                    0
carrier_group_new         0
aircraft_group            0
aircraft_type             0
aircraft_config           0
distance_group            0
class                     0
data_source               0
payload_per_trip        144
seats_per_trip          150
passengers_per_trip     150
dtype: int64

In [171]:
# ok to fill with zero because these did not have departures
pass_cleaned['payload_per_trip'].fillna(0, inplace=True)
pass_cleaned['seats_per_trip'].fillna(0, inplace=True)
pass_cleaned['passengers_per_trip'].fillna(0, inplace=True)

In [172]:
pass_cleaned = pass_cleaned.drop(['unique_carrier_name','payload', 'seats', 'passengers','data_source'], axis=1)

In [173]:
pass_cleaned['origin_airport_id'].unique()

array([10397, 11057, 11292, 11298, 12266, 12892, 13930, 14107, 14747,
       14771])

In [174]:
pass_cleaned

,origin_airport_id,month,year,unique_carrier,dest_airport_id,departures_performed,freight,mail,ramp_to_ramp,region,carrier_group_new,aircraft_group,aircraft_type,aircraft_config,distance_group,class,payload_per_trip,seats_per_trip,passengers_per_trip
105038,10397,2,2018,9E,10135,16,0,0,1818,D,2,6,638,1,2,F,21502.000000,76.000000,62.812500
75521,10397,3,2018,9E,10135,1,0,0,114,D,2,6,638,1,2,F,21502.000000,76.000000,60.000000
132527,10397,4,2018,9E,10135,54,0,0,6397,D,2,6,638,1,2,F,21502.000000,75.981481,66.981481
133856,10397,5,2018,9E,10135,56,0,0,6900,D,2,6,638,1,2,F,21502.000000,76.000000,68.517857
102779,10397,6,2018,9E,10135,13,0,0,1646,D,2,6,638,1,2,F,21502.000000,76.000000,68.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153499,14771,8,2019,TK,16858,31,724475,0,0,I,0,6,637,1,14,F,121500.548387,349.000000,318.935484
159033,14771,9,2019,TK,16858,30,543495,0,0,I,0,6,637,1,14,F,121500.000000,349.000000,318.066667
156498,14771,10,2019,TK,16858,31,608795,0,0,I,0,6,637,1,14,F,121499.967742,349.000000,309.709677
158169,14771,11,2019,TK,16858,30,534598,0,0,I,0,6,637,1,14,F,121500.000000,349.000000,314.866667


In [ ]:
"""
Only flight for WN that tracked fuel was airline_id = 19393 (57 records in fuel_consumption...183181 records in passengers)
"""

In [37]:
df_passengers.isnull().sum()

departures_scheduled     0
departures_performed     0
payload                  0
seats                    0
passengers               0
freight                  0
mail                     0
distance                 0
ramp_to_ramp             0
air_time                 0
unique_carrier           0
airline_id               0
unique_carrier_name      0
region                   0
carrier                  0
carrier_name             0
carrier_group            0
carrier_group_new        0
origin_airport_id        0
origin_city_market_id    0
origin                   0
origin_city_name         0
origin_country           0
origin_country_name      0
dest_airport_id          0
dest_city_market_id      0
dest                     0
dest_city_name           0
dest_country             0
dest_country_name        0
aircraft_group           0
aircraft_type            0
aircraft_config          0
year                     0
month                    0
distance_group           0
class                    0
d

In [7]:
df_passengers['departures_difference'] = df_passengers['departures_performed'] - df_passengers['departures_scheduled']

In [11]:
df_passengers.drop(['departures_scheduled'], axis=1)

,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,...,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source,departures_difference
0,4,387842,1188,649,100745,0,3265,0,0,BA,...,United States,6,637,1,2019,1,7,F,IF,4
1,30,2877549,8560,7250,632027,0,3265,0,0,BA,...,United States,8,819,1,2019,1,7,F,IF,30
2,25,1885730,5350,4177,434705,0,3634,0,0,BA,...,United States,6,887,1,2019,1,8,F,IF,25
3,4,301946,864,611,67885,0,3634,0,0,BA,...,United States,6,889,1,2019,1,8,F,IF,4
4,31,3343991,10323,8031,624070,0,4671,0,0,BA,...,United States,8,819,1,2019,1,10,F,IF,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961566,5,426712,1080,736,135377,0,4211,0,0,BA,...,United States,6,889,1,2019,1,9,F,IF,5
961567,1,101605,219,156,22625,0,4915,0,0,BA,...,United States,6,627,1,2019,1,10,F,IF,1
961568,29,3354604,8613,6448,575848,0,4915,0,0,BA,...,United States,6,637,1,2019,1,10,F,IF,29
961569,17,1293142,3638,2391,206048,0,4182,0,0,BA,...,United States,6,887,1,2019,1,9,F,IF,17


#### These next few cells are just understanding what is in the data

In [51]:
df_passengers['carrier'].unique()

array(['BA', 'BR', 'BW', 'BX', 'C8', 'CA', 'CAZ', 'CC', 'CI', 'CK', 'CM',
       'WS', 'CRV', 'CV', 'CX', 'CZ', 'D7', 'D8', 'DE', 'DI', 'DY', 'E7',
       'EI', 'EK', 'ET', 'EW', 'EY', 'FC', 'FI', 'FJ', 'YX', '07Q', 'AC',
       'IFQ', 'KV', 'KX', 'NLQ', 'OS', 'PP', 'PR', 'SA', 'SQ', 'TS', 'JQ',
       'UA', 'SEB', 'VS', 'EM', '04Q', '1EQ', '9S', '0Q', '1LQ', '1XQ',
       '2UQ', 'AD', 'FYQ', '5Y', 'CP', 'G4', 'HA', 'QX', 'SU', 'LY', 'MF',
       'PS', 'UO', 'VR', 'WG', 'MU', '22Q', '27Q', 'LO', 'LF', '4O',
       '36Q', 'AR', '3AQ', 'HBQ', '0WQ', '1SQ', '1WQ', 'AMQ', 'NEW', 'WI',
       'WN', 'X9', 'Z3', 'NC', '1YQ', '2E', '3F', '7H', '8E', 'KS', 'U7',
       'G7', 'KLQ', 'DL', 'V8', 'K3', '14Q', 'LV', '26Q', '6R', '9V',
       'O6', 'Q6', 'RV', 'TK', 'VIQ', 'VJT', 'Y4', '3SD', 'KZ', 'LH',
       '8V', '2O', '4W', 'GV', 'J5', 'KD', 'N8', 'Q5', 'SNK', 'X4', 'PY',
       'H6', 'L2', 'SY', 'AS', 'AX', 'AA', 'S4', 'SE', 'SK', 'SN', 'SV',
       'SVW', 'TA', 'TN', 'TP', 'TQQ', 'TR', 'UP', 

In [12]:
df_passengers['region'].unique()

array(['I', 'L', 'D', 'P', 'A'], dtype=object)

In [13]:
df_passengers['carrier_group'].unique()

array([0, 3, 1, 2])

In [14]:
df_passengers['carrier_group_new'].unique()

array([0, 3, 6, 5, 4, 2, 1, 9])

In [16]:
df_passengers['origin'].nunique()

2147

In [19]:
df_passengers['origin_city_market_id'].nunique()

1895

In [18]:
df_passengers['origin_country'].nunique()

142

In [21]:
df_passengers['dest'].nunique()

2177

In [22]:
df_passengers['dest_city_market_id'].nunique()

1924

In [23]:
df_passengers['dest_country'].nunique()

153

In [24]:
df_passengers['aircraft_group'].unique()

array([6, 8, 7, 4, 0, 1, 3, 5, 2])

In [25]:
df_passengers['aircraft_type'].unique()

array([637, 819, 887, 889, 627, 882, 644, 820, 614, 694, 699, 696, 682,
       775, 836, 683, 612, 633, 839, 678, 698, 821, 838, 687, 626, 619,
       622, 623, 673, 617, 658, 669, 659, 668, 359, 874, 693, 888, 634,
       456, 620, 441, 479, 646, 616, 770, 688, 725, 609, 625, 721, 608,
       482, 663,  35, 194, 416, 515, 750, 393, 788, 674, 636, 684, 685,
       714, 871, 364, 368, 362, 366, 370, 325,  33, 667, 671, 641, 651,
        79, 632, 635, 740, 131, 489,  40, 483, 415, 406, 405, 412, 340,
       355, 360, 432, 715, 655, 681, 640, 631, 638, 530, 656, 624,  42,
       117,  10, 575, 690, 880, 877, 722,  34,  26,  36, 723, 837, 556,
       675, 697, 873, 442, 628, 485, 686, 648, 218, 459, 677, 824, 150,
       629, 410, 458, 833, 647, 495, 774, 639, 676, 691, 730, 732, 692,
       110, 339,  91, 404, 195,  30, 403, 133, 409, 125, 461, 455, 817,
       390, 642, 469, 652, 471, 507, 872, 689, 484, 771,  31, 143, 430,
       665, 745, 657, 755, 710, 818, 413, 317, 890, 487, 621, 49

In [26]:
df_passengers['aircraft_config'].unique()

array([1, 2, 3, 4])

In [27]:
df_passengers['distance_group'].unique()

array([ 7,  8, 10, 11,  9,  5,  6,  2, 16,  4, 14, 15, 13, 12,  3,  1, 17,
       19, 18, 20, 21])

In [28]:
df_passengers['class'].unique()

array(['F', 'G', 'L', 'P'], dtype=object)

In [29]:
df_passengers['data_source'].unique()

array(['IF', 'DF', 'IU', 'DU'], dtype=object)

In [ ]:
df_passengers['seats'].values()

## Checking Fuel Consumption DataFrame

#### ---Ultimately we decided this was not relevant enough to keep in dataset so it was removed---

In [43]:
df_fuel_consumption = pd.read_csv('../data/18-19_fuel_com.csv')

In [44]:
df_fuel_consumption.columns

Index(['month', 'airline_id', 'unique_carrier', 'carrier', 'carrier_name',
       'carrier_group_new', 'sdomt_gallons', 'satl_gallons', 'spac_gallons',
       'slat_gallons', 'sint_gallons', 'ts_gallons', 'tdomt_gallons',
       'tint_gallons', 'total_gallons', 'sdomt_cost', 'satl_cost', 'spac_cost',
       'slat_cost', 'sint_cost', 'ts_cost', 'tdomt_cost', 'tint_cost',
       'total_cost', 'year'],
      dtype='object')

In [46]:
df_fuel_consumption.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,21161,09Q,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",2,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,1831212,1618745.0,3449957,2019
1,1,21352,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2019
2,1,21492,1BQ,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,102795,74193.0,176988,2019
3,1,21652,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0,0,0.0,0,2019
4,1,21712,2HQ,2HQ,Elite Airways LLC,1,66815.0,0.0,0.0,0.0,...,175499,0.0,0.0,0.0,0.0,175499,1518919,0.0,1518919,2019


In [56]:
df_fuel_consumption.isnull().sum()

month                0
airline_id           0
unique_carrier       0
carrier              0
carrier_name         0
carrier_group_new    0
sdomt_gallons        1
satl_gallons         1
spac_gallons         1
slat_gallons         1
sint_gallons         1
ts_gallons           1
tdomt_gallons        3
tint_gallons         1
total_gallons        6
sdomt_cost           0
satl_cost            1
spac_cost            1
slat_cost            1
sint_cost            1
ts_cost              0
tdomt_cost           0
tint_cost            1
total_cost           0
year                 0
dtype: int64

In [62]:
df_fuel_consumption.loc[df_fuel_consumption['total_gallons'].isnull()]

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
41,1,20422,SY,SY,Sun Country Airlines d/b/a MN Airlines,2,3603825.0,0.0,0.0,1228302.0,...,7127230,0.0,0.0,2630245.0,2630245.0,9757475,9034510,2861737.0,11896247,2019
639,4,20108,NC,NC,Northern Air Cargo Inc.,1,NaN,NaN,NaN,NaN,...,736103,NaN,NaN,NaN,NaN,736103,951577,NaN,970195,2018
896,9,20422,SY,SY,Sun Country Airlines d/b/a MN Airlines,2,2583697.0,0.0,0.0,95011.0,...,6193801,0.0,0.0,247840.0,247840.0,6441641,9354414,355233.0,9709647,2018
947,10,20422,SY,SY,Sun Country Airlines d/b/a MN Airlines,2,3370039.0,0.0,0.0,159654.0,...,8366853,0.0,0.0,422069.0,422069.0,8788922,11395239,661534.0,12056773,2018
998,11,20422,SY,SY,Sun Country Airlines d/b/a MN Airlines,2,3488848.0,0.0,0.0,181639.0,...,8120933,0.0,0.0,442293.0,442293.0,8563226,11027545,674788.0,11702333,2018
1048,12,20422,SY,SY,Sun Country Airlines d/b/a MN Airlines,2,4308507.0,0.0,0.0,677633.0,...,8713955,0.0,0.0,1406377.0,1406377.0,10120332,10234950,1492741.0,11727691,2018


In [71]:
df_fuelcon_WN = df_fuel_consumption[(df_fuel_consumption['carrier'] == 'WN') | (df_fuel_consumption['unique_carrier'] == 'WN')]

In [72]:
df_fuelcon_WN.isnull().sum()

month                0
airline_id           0
unique_carrier       0
carrier              0
carrier_name         0
carrier_group_new    0
sdomt_gallons        0
satl_gallons         0
spac_gallons         0
slat_gallons         0
sint_gallons         0
ts_gallons           0
tdomt_gallons        0
tint_gallons         0
total_gallons        0
sdomt_cost           0
satl_cost            0
spac_cost            0
slat_cost            0
sint_cost            0
ts_cost              0
tdomt_cost           0
tint_cost            0
total_cost           0
year                 0
dtype: int64

In [77]:
df_fuelcon_WN.sort_values(['year', 'month'])

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
506,1,19393,WN,WN,Southwest Airlines Co.,3,161138026.0,0.0,0.0,1834713.0,...,315017449,0.0,0.0,4296170.0,4296170.0,319313619,315397696,4296170.0,319693866,2018
555,2,19393,WN,WN,Southwest Airlines Co.,3,145012888.0,0.0,0.0,1842002.0,...,291089205,0.0,0.0,4363233.0,4363233.0,295452438,291461320,4363233.0,295824553,2018
605,3,19393,WN,WN,Southwest Airlines Co.,3,176770081.0,0.0,0.0,2506139.0,...,345317146,0.0,0.0,5805615.0,5805615.0,351122761,345962204,5805615.0,351767819,2018
655,4,19393,WN,WN,Southwest Airlines Co.,3,174522477.0,0.0,0.0,1928657.0,...,348976261,0.0,0.0,4712631.0,4712631.0,353688892,349419128,4712631.0,354131759,2018
705,5,19393,WN,WN,Southwest Airlines Co.,3,179255111.0,0.0,0.0,2389534.0,...,379119974,0.0,0.0,6220628.0,6220628.0,385340602,379542553,6220628.0,385763181,2018
755,6,19393,WN,WN,Southwest Airlines Co.,3,181821721.0,0.0,0.0,2965799.0,...,394590744,0.0,0.0,7725539.0,7725539.0,402316283,395344788,7725539.0,403070327,2018
805,7,19393,WN,WN,Southwest Airlines Co.,3,188793142.0,0.0,0.0,3010578.0,...,401007391,0.0,0.0,7786048.0,7786048.0,408793439,401635906,7786048.0,409421954,2018
855,8,19393,WN,WN,Southwest Airlines Co.,3,177342030.0,0.0,0.0,1929501.0,...,384095562,0.0,0.0,4971658.0,4971658.0,389067220,384841985,4971658.0,389813643,2018
905,9,19393,WN,WN,Southwest Airlines Co.,3,162637319.0,0.0,0.0,1341792.0,...,348297781,0.0,0.0,3591438.0,3591438.0,351889219,348776993,3591438.0,352368431,2018
955,10,19393,WN,WN,Southwest Airlines Co.,3,175167101.0,0.0,0.0,1525109.0,...,385415886,0.0,0.0,4290762.0,4290762.0,389706648,385961453,4290762.0,390252215,2018


In [81]:
df_fuel_consumption_all = pd.read_csv('../data/fuel_comsumption.csv')

In [82]:
df_fuelcon_WNall = df_fuel_consumption_all[(df_fuel_consumption_all['carrier'] == 'WN') | (df_fuel_consumption_all['unique_carrier'] == 'WN')]

In [83]:
df_fuelcon_WNall.sort_values(['year', 'month'])

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
1371,1,19393.0,WN,WN,Southwest Airlines Co.,3,142391393.0,0.0,0.0,807870.0,...,277084552,0.0,0.0,1556082.0,1556082.0,278640634,277572951,1556082.0,279129033,2015
1427,2,19393.0,WN,WN,Southwest Airlines Co.,3,127521180.0,0.0,0.0,772166.0,...,246070047,0.0,0.0,1564236.0,1564236.0,247634283,246517847,1564236.0,248082083,2015
1483,3,19393.0,WN,WN,Southwest Airlines Co.,3,161798302.0,0.0,0.0,993128.0,...,303973670,0.0,0.0,2069882.0,2069882.0,306043552,304548604,2069882.0,306618486,2015
1538,4,19393.0,WN,WN,Southwest Airlines Co.,3,159318951.0,0.0,0.0,986370.0,...,292154670,0.0,0.0,1992145.0,1992145.0,294146815,292675472,1992145.0,294667617,2015
1593,5,19393.0,WN,WN,Southwest Airlines Co.,3,162319826.0,0.0,0.0,1099019.0,...,326648518,0.0,0.0,2461691.0,2461691.0,329110209,327303482,2461691.0,329765173,2015
1649,6,19393.0,WN,WN,Southwest Airlines Co.,3,167617869.0,0.0,0.0,1288322.0,...,329546985,0.0,0.0,2840242.0,2840242.0,332387227,330197128,2840242.0,333037370,2015
1704,7,19393.0,WN,WN,Southwest Airlines Co.,3,176207727.0,0.0,0.0,1399229.0,...,334398287,0.0,0.0,2888364.0,2888364.0,337286651,334978520,2888364.0,337866884,2015
1759,8,19393.0,WN,WN,Southwest Airlines Co.,3,163921515.0,0.0,0.0,927313.0,...,288106873,0.0,0.0,1727193.0,1727193.0,289834066,288684674,1727193.0,290411867,2015
1813,9,19393.0,WN,WN,Southwest Airlines Co.,3,150235332.0,0.0,0.0,609308.0,...,259602871,0.0,0.0,1104824.0,1104824.0,260707695,260093262,1104824.0,261198086,2015
1867,10,19393.0,WN,WN,Southwest Airlines Co.,3,157094706.0,0.0,0.0,877350.0,...,256156825,0.0,0.0,1595601.0,1595601.0,257752426,256733693,1595601.0,258329294,2015
